# cTWAS pipeline
This module provide an interface for multigroup cTWAS (causal TWAS) analysis with multi-context twas results across all regions.

```
Zhao S, Crouse W, Qian S, Luo K, Stephens M, He X. Adjusting for genetic confounders in transcriptome-wide association studies improves discovery of risk genes of complex traits. Nat Genet (2024). https://doi.org/10.1038/s41588-023-01648-9
```
#### Main Steps
1. Harmonize weights, gwas against LD reference panel 
2. Pre-process input data formats for cTWAS input data format. 
3. Perform cTWAS analysis with LD

## Input
- **LD meta data**: meta data from LD processing with column names of `#chrom`, `start`, `end`, `path`. 
- **regions data**: dataframe of meta data for LD block information with column names of `chr`, `start`, `stop`. 
- **xqtl_region_data**: a dataframe with regions data and file path of the corresponding `refined_twas_weights_data`(*.twas.rds) data from twas pipeline output 
- **gwas_meta_file**: a dataframe with GWAS summary statistics data file paths by chromosome. 

## Output
- A dataframe of cTWAS fine-mapping results for SNP and genes.

## Example
```
sos run ~/githubrepo/xqtl-protocol/code/pecotmr_integration/ctwas.ipynb ctwas \
--cwd /mnt/vast/hpc/csg/cl4215/mrmash/workflow/ \
--xqtl_region_data /mnt/vast/hpc/csg/cl4215/mrmash/workflow/susie_twas/regional_xqtl_data.tsv \
--regions /mnt/vast/hpc/csg/cl4215/mrmash/workflow/twas_mr/pipeline/EUR_LD_blocks_test.bed \
--ld_meta_data /mnt/vast/hpc/csg/data_public/20240409_ADSP_LD_matrix/ld_meta_file.tsv \
--gwas_meta_file /mnt/vast/hpc/csg/cl4215/mrmash/workflow/GWAS/gwas_meta.tsv
```

In [ ]:
[global]
parameter: cwd = path("output/")
parameter: ld_meta_data = path()
parameter: gwas_meta_file = path()
# region info for the input refined_twas_weights_data
parameter: xqtl_region_data = path()
# ld region for the input data
parameter: regions = path()
parameter: name = f"{xqtl_region_data:bn}"
parameter: container = ''
parameter: entrypoint= ('micromamba run -a "" -n' + ' ' + re.sub(r'(_apptainer:latest|_docker:latest|\.sif)$', '', container.split('/')[-1])) if container else ""
parameter: job_size = 100
parameter: walltime = "3h"
parameter: mem = "20G"
parameter: numThreads = 1
import os
import pandas as pd
from collections import OrderedDict

def check_required_columns(df, required_columns):
    """Check if the required columns are present in the dataframe."""
    missing_columns = [col for col in required_columns if col not in list(df.columns)]
    if missing_columns:
        raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
required_xqtl_region_data_columns = ['chrom','start','end','file_path']
required_ld_columns = ['chr', 'start', 'stop']

In [ ]:
[ctwas_1]
# this step we load & format input data for a ld region
regions_df = pd.read_csv(regions, sep="\t",skipinitialspace=True)
regions_df.columns = [col.strip() for col in regions_df.columns] 
regions_df['chr'] = regions_df['chr'].str.strip()
xqtl_region_data = pd.read_csv(xqtl_region_data, sep="\t")
#check for required columns
check_required_columns(regions_df, required_ld_columns)
check_required_columns(xqtl_region_data, required_xqtl_region_data_columns)

# Create a dictionary to map each LD region to its corresponding xQTL file paths
region_xqtl_dict = OrderedDict()
for _, region in regions_df.iterrows():
    region_id = f"{region['chr']}_{region['start']}_{region['stop']}"
    matching_files = xqtl_region_data[
        (xqtl_region_data['chrom'] == region['chr']) & (xqtl_region_data['start'] == region['start']) & 
        (xqtl_region_data['end'] == region['stop'])]['file_path'].tolist()
    region_xqtl_dict[region_id] = matching_files
# Generate inputs for the next steps
region_files = [file for files in region_xqtl_dict.values() for file in files]
region_ids = [region_id for region_id, files in region_xqtl_dict.items() for file in files]
region_ids_str = ','.join(f'"{region_id}"' for region_id in region_ids)

if len(region_files) != len(region_ids):
    raise ValueError("Mismatch between region_files and region_ids lengths")

input: region_files, paired_with=['region_ids_str']
output: f'{cwd:a}/{step_name}/{name}_ctwas.rds', f'{cwd:a}/{step_name}/{name}_weights.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output[0]:n}.stdout", stderr = f"{_output[0]:n}.stderr", container = container, entrypoint = entrypoint

    library(IRanges)
    library(R6)
    library(data.table)
    devtools::load_all("/home/cl4215/githubrepo/pecotmr/") #library(pecotmr)
    devtools::load_all("/home/cl4215/githubrepo/ctwas_multigroup/ctwas/")#library(ctwas)
  
    # Step1: Harmonize weights and gwas
    twas_weights_data <- lapply(c(${_input:r,}), readRDS)
    post_qc_data <- lapply(twas_weights_data, function(twas_data){
                      harmonize_twas(twas_data, "${ld_meta_data}", "${gwas_meta_file}", refined_twas_weights_loader, scale_weights=TRUE)
                    })
    gwas_studies <- unique(names(find_data(post_qc_data, c(3, "gwas_qced"))))
    saveRDS(post_qc_data, ${_output[1]:r})
  
    # Step2: Preprocess weights, LD variants data. 
    weights <- do.call(c, lapply(post_qc_data, function(twas_data){
                  get_ctwas_weights(twas_data, "${ld_meta_data}")# reshape weights for all gene-context pairs in the region for cTWAS analysis
              }))
    weights <- weights[!sapply(weights, is.null)]
    
    # get region_info and snp_info: LD block meta info and variant - bim file data.
    region_of_interest <- region_to_df(c(${region_ids_str}))
    meta_data <- get_ctwas_meta_data("${ld_meta_data}", "${regions}")
    region_info <- meta_data$region_info
    LD_info <- meta_data$LD_info
        
    # load LD variants
    bim_file_paths <- unique(do.call(c, lapply(1:nrow(region_of_interest), function(region_row){
                          get_regional_ld_meta("${ld_meta_data}", region_of_interest[region_row,,drop=FALSE])$intersections$bim_file_paths})))
    snp_info <- lapply(bim_file_paths, function(file){
                       bimfile <- read.table(file, header = FALSE, sep="\t")[, c(1,2,4:8)]# original colnames: "chrom", "variants", "GD", "pos", "A1", "A2", "variance", "allele_freq", "n_nomiss"
                       bimfile$V2 <- gsub("chr", "", gsub("_", ":", bimfile$V2))
                       colnames(bimfile) <- c("chrom", "id", "pos", "alt", "ref", "variance", "allele_freq") # A1:alt, A2: ref
                       return(bimfile)})
    names(snp_info)<- do.call(c, lapply(bim_file_paths, function(x) { parts <- strsplit(basename(x), "[_:/.]")[[1]][1:3]
                            gsub("chr", "", paste(parts, collapse="_"))}))
    
    # Step3: Simple cTWAS with LD for all regions
    ctwas_res <- list()
    for (study in gwas_studies){
      gwas_z <- do.call(rbind, lapply(post_qc_data, function(x) find_data(x, c(2, "gwas_qced", study), docall=rbind)))
      colnames(gwas_z)[which(colnames(gwas_z)=="variant_id")] <- "id"
      z_snp <- gwas_z[, c("id", "A1", "A2", "z")]
      z_snp <- z_snp[!duplicated(z_snp$id), ]
      # compute gene_z for with each study z_snp 
      z_gene <- compute_gene_z(z_snp, weights, ncore = ${numThreads}, logfile = file.path(${_output[0]:dr}, paste0("${name}", ".compute_gene_z.log")))
      weights_update <- weights[names(weights)[!names(weights) %in% z_gene$id[is.na(z_gene$z)]]] # remove gene-context pair with NA result
      z_gene  <- z_gene[!is.na(z_gene$z),] 
      ctwas_res[[study]] <- ctwas_res <- ctwas_sumstats(z_snp,
                              weights_update,
                              region_info,
                              snp_info,
                              LD_info,
                              z_gene = z_gene,
                              thin = 1,
                              ncore = ${numThreads},
                              outputdir = ${_output[0]:dr},
                              outname = "${name}",
                              logfile = file.path(${_output[0]:dr}, paste0("${name}", ".ctwas_sumstats.log")),
                              verbose = FALSE, 
                              cor_dir = NULL,
                              save_cor = TRUE,
                              screen_method="nonSNP_PIP",
                              LD_format="custom", 
                              LD_loader_fun = ctwas_ld_loader)
    }

    # Step4: save results 
    saveRDS(ctwas_res, ${_output[0]:r})